In [ ]:
import json
from fuzzywuzzy import fuzz
from collections import defaultdict
from tabulate import tabulate

In [ ]:
# Chargement des données
with open("transcriptions_ordonnées.json", encoding="utf-8") as f:
    transcriptions = json.load(f)

with open("dic_final.json", encoding="utf-8") as f:
    dictionnaire_lemmes = json.load(f)

In [1]:
# Création du dictionnaire : forme -> liste de lemmes triés par nombre de variantes (desc)
forme_vers_lemmes_raw = defaultdict(list)
for lemme, variantes in dictionnaire_lemmes.items():
    for variante in variantes:
        forme_vers_lemmes_raw[variante.lower()].append((lemme, len(variantes)))

forme_vers_lemmes = {
    forme: [lemme for lemme, _ in sorted(lemmes, key=lambda x: -x[1])]
    for forme, lemmes in forme_vers_lemmes_raw.items()
}

def trouver_lemmes(mot):
    return forme_vers_lemmes.get(mot.strip(".,!?").lower(), [])

def trouver_meilleur_alignement(mot, candidats, indices_utilisés, seuil=80):
    lemmes_mot = trouver_lemmes(mot)
    for lemme in lemmes_mot:
        for i, cand in enumerate(candidats):
            if i in indices_utilisés:
                continue
            lemmes_cand = trouver_lemmes(cand)
            if lemmes_cand and lemmes_cand[0] == lemme:
                return cand, i
    meilleur_score = 0
    meilleur_mot = None
    meilleur_index = None
    for i, cand in enumerate(candidats):
        if i in indices_utilisés:
            continue
        score = fuzz.ratio(mot.lower(), cand.lower())
        if score > meilleur_score and score >= seuil:
            meilleur_score = score
            meilleur_mot = cand
            meilleur_index = i
    return (meilleur_mot, meilleur_index) if meilleur_mot else ("—", None)

def alignement_deux_listes(l1, l2, seuil=80):
    tuples_alignés = []
    utilisés_l1 = set()
    utilisés_l2 = set()

    toutes = [(mot, 'man', i) for i, mot in enumerate(l1)] + \
             [(mot, 'corr', i) for i, mot in enumerate(l2)]

    toutes_triées = sorted(toutes, key=lambda x: x[2])  # garder ordre local

    for mot, origine, index in toutes_triées:
        if origine == 'man' and index in utilisés_l1:
            continue
        if origine == 'corr' and index in utilisés_l2:
            continue

        man, corr = "—", "—"
        if origine == 'man':
            man = mot
            utilisés_l1.add(index)
            corr, i2 = trouver_meilleur_alignement(mot, l2, utilisés_l2, seuil)
            if i2 is not None:
                utilisés_l2.add(i2)
        else:
            corr = mot
            utilisés_l2.add(index)
            man, i1 = trouver_meilleur_alignement(mot, l1, utilisés_l1, seuil)
            if i1 is not None:
                utilisés_l1.add(i1)

        tuples_alignés.append((man, corr))

    return tuples_alignés

from tabulate import tabulate  # Dodaj import na górze!

# Structure pour les résultats à sauvegarder
resultats = []

for exemple in transcriptions:
    num = exemple["numéro d'apprenant"]
    pays = exemple.get("pays", "Unknown")

    apprenant_data = {
        "apprenant": str(num),
        "pays": pays,
        "segments": []
    }

    i = 1
    while f"transcription manuelle ({i})" in exemple:
        man = exemple.get(f"transcription manuelle ({i})", "").split()
        corr = exemple.get(f"version correcte en polonais ({i})", "").split()

        if not any([man, corr]):
            i += 1
            continue

        alignements = alignement_deux_listes(man, corr)

        segment_data = {
            "segment_id": i,
            "segments": [
                {"manuel": m, "correct": c}
                for m, c in alignements
            ]
        }

        # Tableaux
        #print(f"\nApprenant: {num} | Pays: {pays} | Segment {i}")
        table = [[s["manuel"], s["correct"]] for s in segment_data["segments"]]
        #print(tabulate(table, headers=["Manuel", "Correct"], tablefmt="grid"))

        apprenant_data["segments"].append(segment_data)
        i += 1

    resultats.append(apprenant_data)

# Sauvegarde des résultats
with open("alignement_mots_manuel_correct.json", "w", encoding="utf-8") as f:
    json.dump(resultats, f, ensure_ascii=False, indent=4)

print("\nRésultats sauvegardés dans 'alignement_mots_manuel_correct.json'")


Résultats sauvegardés dans 'alignement_mots_manuel_correct.json'


In [5]:
# Statistiques d'alignement
mots_alignés = 0
mots_total = 0

for apprenant in resultats:
    for segment in apprenant["segments"]:
        for paire in segment["segments"]:
            if paire["manuel"] != "—" and paire["correct"] != "—":
                mots_alignés += 1
            mots_total += 1

pourcentage = (mots_alignés / mots_total * 100) if mots_total > 0 else 0

print(f"\nNombre total de mots des transcriptions manuelles : {mots_total}")
print(f"Nombre de mots alignés : {mots_alignés}")
print(f"Pourcentage d'alignement : {pourcentage:.2f} %")



Nombre total de mots des transcriptions manuelles : 4178
Nombre de mots alignés : 3466
Pourcentage d'alignement : 82.96 %
